In [45]:
import numpy as np
import random
import copy
def activation(x):
    return np.where(x < 0.5, 0, 1)
def mlp(X, y, w1, b1, w2, b2):
    # Forward propagation
    z1 = np.dot(X, w1) + b1
    a1 = activation(z1)
    z2 = np.dot(a1, w2) + b2
    y_pred = activation(z2)
    
    # Calculate the loss (mean squared error)
    mse = np.mean((y - y_pred) ** 2)
    
    return mse



In [46]:
class PSO:
    def __init__(self, n_particles, dimensions, max_iterations, c1, c2, w, X, y):
        self.n_particles = n_particles
        self.dimensions = dimensions
        self.max_iterations = max_iterations
        self.c1 = c1
        self.c2 = c2
        self.w = w
        self.X = X
        self.y = y
        self.particles = []
        self.velocities = []
        self.personal_bests = []
        self.global_best = None
        self.global_best_fitness = float('inf')
        
        # Initialize particles and velocities
        for i in range(self.n_particles):
            particle = np.random.uniform(-1, 1, self.dimensions)
            velocity = np.zeros(self.dimensions)
            self.particles.append(particle)
            self.velocities.append(velocity)
            fitness = mlp(self.X, self.y, particle[0:6].reshape(2, 3), particle[6:9].reshape(1, 3), particle[9:12].reshape(3, 1), particle[12:13].reshape(1, 1))
            self.personal_bests.append((particle, fitness))
            
            if fitness < self.global_best_fitness:
                self.global_best = particle
                self.global_best_fitness = fitness
                
    def update(self):
        for i in range(self.n_particles):
            # Update velocity
            r1 = np.random.uniform(0, 1, self.dimensions)
            r2 = np.random.uniform(0, 1, self.dimensions)
            self.velocities[i] = self.w * self.velocities[i] + self.c1 * r1 * (self.personal_bests[i][0] - self.particles[i]) + self.c2 * r2 * (self.global_best - self.particles[i])
            
            # Update particle
            self.particles[i] = self.particles[i] + self.velocities[i]
            
            # Clip particle values to [-1, 1]
            self.particles[i] = np.clip(self.particles[i], -1, 1)
            
            # Calculate fitness
            fitness = mlp(self.X, self.y, self.particles[i][0:6].reshape(2, 3), self.particles[i][6:9].reshape(1, 3), self.particles[i][9:12].reshape(3, 1), self.particles[i][12:13].reshape(1, 1))
                    # Update personal best
        if fitness < self.personal_bests[i][1]:
            self.personal_bests[i] = (self.particles[i], fitness)
            
            # Update global best
            if fitness < self.global_best_fitness:
                self.global_best = self.particles[i]
                self.global_best_fitness = fitness
                
    def train(self):
      for i in range(self.max_iterations):
        self.update()
      #print(self.global_best)  
      # Return the global best particle and its fitness
      return self.global_best, self.global_best_fitness


In [50]:
# Input data
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
# Target output
y = np.array([[0], [1], [1], [0]])

# PSO hyperparameters
n_particles = 50
dimensions = 13
max_iterations = 1000
w = 0.45
c1 = 1.4
c2 = 1.3

# Train MLP using PSO
pso = PSO(n_particles, dimensions, max_iterations, c1, c2, w, X, y)
best_particle, best_fitness = pso.train()

# Extract MLP weights and biases from best particle
w1 = best_particle[0:6].reshape(2, 3)
b1 = best_particle[6:9].reshape(1, 3)
w2 = best_particle[9:12].reshape(3, 1)
b2 = best_particle[12:13].reshape(1, 1)

# Test MLP on input data
z1 = np.dot(X, w1) + b1
a1 = activation(z1)
z2 = np.dot(a1, w2) + b2
y_pred = activation(z2)

# Print predicted output
print(y_pred)


[[0]
 [1]
 [1]
 [0]]
